# Аналитика данных компании Gectaro

**Цель проекта**  
  
Проанализировать имеющиеся данные, составить портреты наиболее типичных клиентов, разобраться в причинах оттока клиентов, предложить решения по их минимизации. Для этого предлагается определить ключевые признаки для компаний, наличие которых (или их появление) влияет на отмену (отсутствие оплаты) подписки.

## Загрузка данных и подготовка к анализу

### Загрузка библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import numpy as np
import warnings

Уберем ограничение по выводу строк, колонок и символов в записи и включаем игнорирование ошибок Jupiter.

In [2]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

#Игнорируем предупреждения Jupiter
warnings.filterwarnings('ignore')

Прописываем темы визуализации.

In [3]:
sns.set_style('darkgrid')
plt.style.use('ggplot')

### Загрузка данных

Запишем датафрейммы  activities.csv, billing.csv, companies.csv и people.csv в датасеты  activities, billing, companies и people соответственно.

In [4]:
try:
    activities, billing, companies, people = (
        pd.read_csv('data/activities.csv', index_col=0),
        pd.read_csv('data/billing.csv', index_col=0),
        pd.read_csv('data/companies.csv', index_col=0),
        pd.read_csv('data/people.csv', index_col=0)
    )


except:
    activities, billing, companies, people = (
        pd.read_csv('activities.csv', index_col=0),
        pd.read_csv('billing.csv', index_col=0),
        pd.read_csv('companies.csv', index_col=0),
        pd.read_csv('people.csv', index_col=0)
    )

Подготовим функцию для вывода информации о датафрейме.

In [5]:
def get_info(df):
    """
    Функция для вывода информации о датафрейме
    
    Args:
        df: датафрейм 
        
    Returns:
        print(информация о датафрейме)
        display(первые 5 строк датафрейма).
    """

    print(df.info())
    display(df.head())

Посмотрим информацию о датафреймах.

In [6]:
get_info(activities)

<class 'pandas.core.frame.DataFrame'>
Index: 6316 entries, 0 to 6556
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   company_id                     6316 non-null   int64  
 1   month                          6316 non-null   object 
 2   company-month-billing          6316 non-null   float64
 3   company-month-stocks           6316 non-null   float64
 4   company-month-backups          6316 non-null   float64
 5   company-month-projects         6316 non-null   float64
 6   company-month-contractors      6316 non-null   float64
 7   company-month-resource_orders  6316 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 444.1+ KB
None


,company_id,month,company-month-billing,company-month-stocks,company-month-backups,company-month-projects,company-month-contractors,company-month-resource_orders
0,1,2019-09-01 00:00:00+00:00,1.0,0.0,0.0,7.0,0.0,0.0
1,1,2019-12-01 00:00:00+00:00,1.0,0.0,0.0,2.0,1.0,0.0
2,1,2020-04-01 00:00:00+00:00,4.0,1.0,0.0,1.0,6.0,0.0
3,1,2020-07-01 00:00:00+00:00,1.0,7.0,3.0,5.0,0.0,0.0
4,21,2020-01-01 00:00:00+00:00,3.0,0.0,0.0,1.0,0.0,0.0


В нашем распоряжении **activities** датафрейм из 8 колонок и 6316 строк.
- `company_id` -  Идентификатор компании
- `month` -  Отчетный месяц
- `company-month-billing` -  Количество счетов, выставленных в приложении, компанией за месяц
- `company-month-stocks` -  Месячное количество складов (аренда, заказы и т.д.), используемых компанией
- `company-month-backups` -  Месячное количество бэкапов компании
- `company-month-projects` -  Месячное количество проектов компании
- `company-month-contractors` -  Месячное количество контрагентов компании
- `company-month-resource_orders` -  Месячное количество заказов материалов компании

In [7]:
get_info(billing)

<class 'pandas.core.frame.DataFrame'>
Index: 3480 entries, 4 to 18953
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               3480 non-null   int64  
 1   id                       3480 non-null   int64  
 2   user_id                  3480 non-null   int64  
 3   company_id               3480 non-null   int64  
 4   billing_tariff_id        3480 non-null   int64  
 5   payed_until              3480 non-null   int64  
 6   is_payed                 3480 non-null   bool   
 7   created_at               3480 non-null   int64  
 8   updated_at               3480 non-null   int64  
 9   billing_tariff_price_id  3480 non-null   float64
 10  comment                  745 non-null    object 
 11  pay_date                 1108 non-null   float64
dtypes: bool(1), float64(2), int64(8), object(1)
memory usage: 329.6+ KB
None


,Unnamed: 0,id,user_id,company_id,billing_tariff_id,payed_until,is_payed,created_at,updated_at,billing_tariff_price_id,comment,pay_date
4,4,3050,567,512,3,1640153302,False,1608617302,1608617302,12.0,NaN,NaN
6,6,3051,2861,2474,7,1640131200,True,1608626162,1608626181,23.0,NaN,1.608626e+09
16,16,3059,1902,1645,2,1616577484,False,1608801484,1608801484,6.0,NaN,NaN
18,18,3065,2352,2035,7,1643500799,False,1608883175,1608883175,23.0,NaN,NaN
24,24,3062,1074,926,3,1611964800,True,1608814072,1608899674,9.0,NaN,1.609200e+09


В нашем распоряжении **billing** датафрейм из 12 колонок и 3480 строк.
- `Unnamed:0` - Старый индекс
- `id` - Неизвестная колонка
- `user_id` - Идентификатор пользователя
- `company_id` - Идентификатор компании пользователя
- `billing_tariff_id` - Тарифный план
- `payed_until` - Дата, до которой подписка компании действует
- `is_payed` - Статус оплаты
- `created_at` - Дата создания
- `updated_at` - Дата обновления
- `billing_tariff_price_id` - id стоимости тарифа
- `comment` - Комментарии
- `pay_date` - Дата платежа

In [8]:
get_info(companies)

<class 'pandas.core.frame.DataFrame'>
Index: 723 entries, 19 to 17972
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       723 non-null    int64  
 1   working_direction        720 non-null    object 
 2   employer_count_interval  723 non-null    float64
 3   created_at               723 non-null    int64  
 4   updated_at               723 non-null    int64  
 5   billing_tariff_id        716 non-null    float64
 6   billing_is_over          723 non-null    bool   
dtypes: bool(1), float64(2), int64(3), object(1)
memory usage: 40.2+ KB
None


,id,working_direction,employer_count_interval,created_at,updated_at,billing_tariff_id,billing_is_over
19,514,-,1.0,1571316635,1670298314,4.0,True
57,383,Строительство,2.0,1564142565,1676550050,7.0,False
69,495,-,1.0,1570535695,1598281714,2.0,False
75,724,-,1.0,1579514393,1670298294,4.0,True
99,471,-,1.0,1569328017,1670298360,7.0,True


В нашем распоряжении **companies** датафрейм из 7 колонок и 723 строк.
- `id` - id компании
- `working_direction` - сфера деятельности
- `employer_count_interval` - id количества сотрудников
- `created_at` - дата создания компании в системе
- `updated_at` - дата обновления информации о компании
- `billing_tariff_id` - id тарифа 
- `billing_is_over` - истекла ли оплата



In [9]:
get_info(people)

<class 'pandas.core.frame.DataFrame'>
Index: 5095 entries, 2 to 17054
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          5095 non-null   int64  
 1   company_id  5095 non-null   int64  
 2   user_id     3813 non-null   float64
 3   role_id     5095 non-null   int64  
 4   created_at  5095 non-null   int64  
 5   updated_at  5095 non-null   int64  
 6   is_deleted  5095 non-null   bool   
 7   deleted_at  2180 non-null   float64
dtypes: bool(1), float64(2), int64(5)
memory usage: 323.4 KB
None


,id,company_id,user_id,role_id,created_at,updated_at,is_deleted,deleted_at
2,6601,495,NaN,7133,1570535962,1570535962,True,1.594067e+09
6,7025,1,NaN,1,1594120484,1594120484,True,1.594120e+09
7,7165,1255,1734.0,18085,1595438102,1595439391,True,1.633960e+09
10,7130,1470,1703.0,1,1595234824,1595234824,False,NaN
17,7163,1492,1731.0,1,1595429211,1598540644,True,1.620195e+09


В нашем распоряжении **people** датафрейм из 8 колонок и 5095 строк.
 - `id`- индекс
 - `company_id`- id компании
 - `user_id`- id пользователя
 - `role_id`- id роли (должности)
 - `created_at`- дата создания пользователя
 - `updated_at`- дата обновления информации о пользователе
 - `is_deleted`- удален ли пользователь
 - `deleted_at`- дата удаления

In [10]:
# activities, billing, companies, people